In [1]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.io import imread
import rasterio as rio
from rasterio.plot import reshape_as_image
import maskSLIC.maskslic as seg
import skimage.segmentation as skseg
from glob import glob 

In [2]:
# Diretórios de output
slic_dir = '/home/jp/Documents/IME/TCC/CODE/Capstone-Project/data/slic/'
output_dir = '/home/jp/Documents/IME/TCC/CODE/Capstone-Project/data/segmentation_image/'

# Diretórios de input
c2_dir = '/home/jp/Documents/IME/TCC/CODE/Capstone-Project/data/c2/'
c2_files = sorted(glob(c2_dir+'*'))
despeckle_dir = '/home/jp/Documents/IME/TCC/CODE/Capstone-Project/data/despeckle/'
despeckle_files = sorted(glob(despeckle_dir+'*'))
colorized_dir = '/home/jp/Documents/IME/TCC/CODE/Capstone-Project/data/colorized/'
colorized_files = sorted(glob(colorized_dir+'*'))
truth_dir = '/home/jp/Documents/IME/TCC/CODE/Capstone-Project/data/truth/'
truth_files = sorted(glob(truth_dir+'ground/*'))

In [3]:
# Funções para realizar a segmentação

def rescale_img(src, w, h):
    data = src.read(
        out_shape=(
            src.count,
            w,
            h
        ),
        )

# scale image transform
    transform = src.transform * src.transform.scale(
        (src.width / data.shape[-1]),
        (src.height / data.shape[-2])
    )

    return data

def test(image, n_segments, enforce_connectivity, distance, spatial_weight, filename='file', max_iter=10, compactness=0.1, color_image=None, w=None, h=None, mask=None):
    # Remover valores iguais a 0 para evitar problemas
    # nas distâncias que usam log
    zero_mask = (image == 0)
    random_numbers = np.random.rand(*image.shape)
    image[zero_mask] = random_numbers[zero_mask]

    segments = seg.slic(image, n_segments=n_segments, max_iter=max_iter, compactness=compactness, 
                       enforce_connectivity=enforce_connectivity, distance=distance, spatial_weight=spatial_weight, mask=mask)
    if color_image is None:
        display_image = image[:, :, 2]/np.max(image[:, :, 2])
    else:
        display_image = color_image
    image_with_bounds = skseg.mark_boundaries(image=display_image, label_img=segments)
    plt.imsave(output_dir + filename + '.png', image_with_bounds)
    np.save(slic_dir + filename + '.npy', segments)
    return segments

def save_truth_mask(full_mask, satellite, i):
    new_array = np.full((full_mask.shape[0], full_mask.shape[1]), 3)

    # Conditions for assigning values based on channels
    # If channel 1 value is 255, assign 1; if channel 2 value is 1, assign 2; else, keep as 3
    new_array[full_mask[:, :, 0] == 255] = 1
    new_array[full_mask[:, :, 1] == 255] = 2
    np.save(truth_dir + f'ae{i}_{satellite}.npy', new_array)

def get_maskslic_mask(full_mask):
    new_array = np.full((full_mask.shape[0], full_mask.shape[1]), 0)

    # Conditions for assigning values based on channels
    # If channel 1 value is 255, assign 1; if channel 2 value is 1, assign 2; else, keep as 3
    new_array[full_mask[:, :, 0] == 255] = 1
    new_array[full_mask[:, :, 1] == 255] = 1
    return new_array
    
def calculate_truths(i, w, h):
    with rio.open(truth_files[i]) as src:
        ground = src.read()
        ground = reshape_as_image(ground)
        scaled_ground = rescale_img(src, w, h)
        scaled_ground = reshape_as_image(scaled_ground)
    save_truth_mask(ground, 's2', i)
    save_truth_mask(scaled_ground, 's1', i)

    return ground[:,:,3], scaled_ground[:,:,3]

In [4]:
# Segmentação pelas distâncias estatísticas
distances = {
    0: 'wishart',
    1: 'rw',
    2: 'snll',
    3: 'hlt'
}
for i in range(len(c2_dir)):
    with rio.open(c2_files[i]) as src:
        c2 = src.read()
        c2 = reshape_as_image(c2)
    _, classes = calculate_truths(i, c2.shape[0], c2.shape[1])
    for dist in distances.keys():
        filename = slic_dir + f'ae{i+2}/area{i+2}_{distances.get(i)}'
        test(image=c2, n_segments=10000, enforce_connectivity=False, distance=dist, spatial_weight=5, filename=filename, max_iter=10, compactness=0.1, color_image=classes)
        test(image=c2, n_segments=10000, enforce_connectivity=False, distance=dist, spatial_weight=5, filename=filename+'_mask', max_iter=10, compactness=0.1, color_image=classes, mask=get_maskslic_mask(classes))


5


TypeError: _slic_cython() got multiple values for keyword argument 'only_dist'

In [5]:
def rescale_img(src, w, h):
    data = src.read(
        out_shape=(
            src.count,
            w,
            h
        ),
        )

# scale image transform
    transform = src.transform * src.transform.scale(
        (src.width / data.shape[-1]),
        (src.height / data.shape[-2])
    )

    return data

def test(filepath, n_segments, enforce_connectivity, distance, spatial_weight, filename='file', max_iter=10, compactness=0.1, color_image=None, w=None, h=None, mask=None):
    with rio.open(filepath) as src:
        if w and h:
            image = rescale_img(src, w, h)
        else:
            image = src.read()
        image = reshape_as_image(image)

        # Remover valores iguais a 0 para evitar problemas
        zero_mask = (image == 0)
        random_numbers = np.random.rand(*image.shape)
        image[zero_mask] = random_numbers[zero_mask]
        print('antes do slic')
        print(type(max_iter))
        vv_slic = seg.slic(image, n_segments=n_segments, max_iter=max_iter, compactness=compactness, enforce_connectivity=True, distance=distance, spatial_weight=spatial_weight, mask=mask)
        print('antes do mark')
        plt.imsave(filename+'.png', vv_slic)
        if color_image is None:
            display_image = image[:, :, 2]
        else:
            display_image = color_image
        image_with_bounds = skseg.mark_boundaries(image=display_image, label_img=vv_slic)
        print('depois do mark')
        plt.imshow(image_with_bounds)
        plt.imsave(filename+'.png', image_with_bounds)
        np.save(slic_dir+filename+'.npy', vv_slic)
        return vv_slic
    
file = '/media/jp/FreeAgent GoFlex Drive/SAR/AREA_3_mat.tif'
filename = 'AREA3_b_20000_snll_sanity_check'
slic = test(file, 20000, True, 2, 5, filename) 

antes do slic
<class 'int'>
5


TypeError: _slic_cython() got multiple values for keyword argument 'only_dist'